In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
results = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/results.csv')
scorers = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/goalscorers.csv')
shootouts = pd.read_csv('/kaggle/input/international-football-results-from-1872-to-2017/shootouts.csv')

In [3]:
results.sample(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
32271,2010-01-04,Egypt,Mali,1,0,Friendly,Dubai,United Arab Emirates,True
29402,2007-03-24,Cameroon,Liberia,3,1,African Cup of Nations qualification,Yaoundé,Cameroon,False
37923,2015-11-10,Vanuatu,Fiji,2,1,Friendly,Port Vila,Vanuatu,False


In [4]:
nb_tournament = results['tournament'].unique().tolist()
print((f'There are {len(nb_tournament)} tournaments in this data'))

There are 147 tournaments in this data


Parsing dates

In [5]:
results['date'] = pd.to_datetime(results['date'], errors = 'coerce',format= '%Y-%m-%d')

results['Year'] = results.date.dt.year
results['Month'] = results.date.dt.month
results['day'] = results.date.dt.day

weekday_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
results['week_day'] = results['date'].map(lambda x: weekday_order[x.weekday()])

results.sample(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,Month,day,week_day
29409,2007-03-24,El Salvador,Honduras,0,2,Friendly,Fort Lauderdale,United States,True,2007,3,24,Sat
20529,1997-01-22,Mexico,Peru,0,0,USA Cup,Pasadena,United States,True,1997,1,22,Wed
247,1909-08-15,Argentina,Uruguay,2,1,Copa Lipton,Buenos Aires,Argentina,False,1909,8,15,Sun


Define winner

In [6]:
def winner(results):
    if results.home_score > results.away_score:
        return results.home_team
    if results.home_score < results.away_score:
        return results.away_team
    else:
        return 'tie'

results['winner'] = results.apply(winner,axis=1)
results.sample(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,Month,day,week_day,winner
21931,1998-10-01,Qatar,Saudi Arabia,1,3,Arab Cup,Doha,Qatar,False,1998,10,1,Thu,Saudi Arabia
30180,2007-11-21,Jamaica,Guatemala,2,0,Friendly,Kingston,Jamaica,False,2007,11,21,Wed,Jamaica
24686,2001-09-08,Uzbekistan,Oman,5,0,FIFA World Cup qualification,Tashkent,Uzbekistan,False,2001,9,8,Sat,Uzbekistan
22678,1999-10-09,Portugal,Hungary,3,0,UEFA Euro qualification,Lisbon,Portugal,False,1999,10,9,Sat,Portugal
810,1924-06-14,Australia,Canada,0,1,Friendly,Sydney,Australia,False,1924,6,14,Sat,Canada


## Results dataframe


In [7]:
nb_matches_year = results.groupby('Year').size().reset_index(name="matches_played").sort_values('Year', ascending = False)

fig = go.Figure()
fig = px.line(nb_matches_year, x= "Year", y = ['matches_played'], labels = {'matches_played' : 'number of matches'})

fig.update_layout(title='Number of matches over the years',
                  yaxis_title='Matches played')

fig.show()

print('Number of matches declined significantly due to Covid in 2020')

Number of matches declined significantly due to Covid in 2020


In [8]:
fig = px.histogram(results, 
                   x = 'Month',
                   text_auto = True,
                   category_orders = {'Month': list(range(1,13))},
                   title = 'Distribution of matches per month')

fig.update_layout(bargap = 0.15)
fig.update_xaxes(type = 'category')
fig.show()

print('March: FIFA-designated matchdays for World Cup qualifiers and other tournaments take place')
print('June: Major tournaments (World Cup, Euro, Copa America) & continental championships take place')
print('Sep to Nov: Another months with international breaks, often used for friendlies and the conclusion of World Cup qualifiers')

March: FIFA-designated matchdays for World Cup qualifiers and other tournaments take place
June: Major tournaments (World Cup, Euro, Copa America) & continental championships take place
Sep to Nov: Another months with international breaks, often used for friendlies and the conclusion of World Cup qualifiers


In [9]:
fig = px.histogram(results,
                   x = 'week_day',
                   text_auto = True,
                   title = 'Distribution of matches per week day')

fig.update_layout(bargap = 0.2)
fig.show()

print('From my perspective, Mid-week and weekends witness most games due to audience considerations, tradition, & logistics')

From my perspective, Mid-week and weekends witness most games due to audience considerations, tradition, & logistics


In [10]:
winner_count = results.groupby('winner').size().reset_index(name = 'number of wins').sort_values('number of wins', ascending = False)
winner_count = winner_count[winner_count['winner'] != "tie"][:10]

fig = px.bar(winner_count, x='winner', y='number of wins',
             text_auto = True,
             title='Number of Wins by Country',
             labels={'number of wins': 'Number of Wins', 'winner': 'Country/Team'})

fig.show()

print('Europe has most Countries regarding number of wins (6), followed by Americas (3), Asia (1)')

Europe has most Countries regarding number of wins (6), followed by Americas (3), Asia (1)


In [11]:
print('Matches with largeset score gap:')
results['gap']=  abs(results['home_score'] - results['away_score'])
results.sort_values('gap', ascending = False)[:5]

Matches with largeset score gap:


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,Month,day,week_day,winner,gap
24160,2001-04-11,Australia,American Samoa,31,0,FIFA World Cup qualification,Coffs Harbour,Australia,False,2001,4,11,Wed,Australia,31
8027,1971-09-13,Tahiti,Cook Islands,30,0,South Pacific Games,Papeete,French Polynesia,False,1971,9,13,Mon,Tahiti,30
11225,1979-08-30,Fiji,Kiribati,24,0,South Pacific Games,Nausori,Fiji,False,1979,8,30,Thu,Fiji,24
24157,2001-04-09,Australia,Tonga,22,0,FIFA World Cup qualification,Coffs Harbour,Australia,False,2001,4,9,Mon,Australia,22
6134,1966-04-03,Libya,Oman,21,0,Arab Cup,Baghdad,Iraq,True,1966,4,3,Sun,Libya,21


In [12]:
total_home_away_score = results.groupby('Year').agg({'home_score': 'sum' , 'away_score' : 'sum'}).reset_index()
total_home_away_score

,Year,home_score,away_score
0,1872,0,0
1,1873,4,2
2,1874,2,1
3,1875,2,2
4,1876,7,0
...,...,...,...
147,2019,1973,1358
148,2020,428,311
149,2021,1798,1193
150,2022,1373,896


In [13]:
fig = px.line(total_home_away_score, x='Year', y =['home_score', 'away_score'],
              title = 'Total Home Score vs Away Score over years')

fig.show()

print('Home-field advantage seems quite correct due to familiar surroundings, supportive crowd and travel fatigue from away teams')

Home-field advantage seems quite correct due to familiar surroundings, supportive crowd and travel fatigue from away teams


In [14]:
results_ = results.copy()

def labels(results_):
    if results_['home_score'] > results_['away_score']:
        return 'Home team win'
    if results_['home_score'] < results_['away_score']:
        return 'Away team win'
    return 'Draw'

results_['outcomes'] = results_.apply(lambda x: labels(x), axis=1)
results_.head()

## Create pie chart

outcome = go.Pie(labels = ['Home team win', 'Away team win', 'Draw'], values = results_['outcomes'].value_counts(),
                 textfont = dict(size = 15), opacity = 0.8)

layout = go.Layout(title= {
    'text':'Match outcomnes proportion between home and away team',
    'xanchor' : 'center',
    'yanchor' : 'top',
    'y' :  0.9 ,
    'x' :  0.5
})

fig = go.Figure(data = [outcome], layout = layout)
fig.show()

Total points (win - 3 pts, draw - 1 pts, lose - 0 pts)

In [15]:
results_['home_points'] = results_.apply(lambda row: 3 if row['outcomes'] == 'Home team win' else 1 if row['outcomes'] == 'Draw' else 0, axis=1)
results_['away_points'] = results_.apply(lambda row: 3 if row['outcomes'] == 'Away team win' else 1 if row['outcomes'] == 'Draw' else 0, axis=1)

home_team_points = results_.groupby('home_team')['home_points'].sum().reset_index().rename(columns={'home_team': 'country', 'home_points': 'total_points'})
away_team_points = results_.groupby('away_team')['away_points'].sum().reset_index().rename(columns={'away_team': 'country', 'away_points': 'total_points'})

total_points = home_team_points.merge(away_team_points, on='country', how='outer', suffixes=('_home', '_away')).fillna(0)
total_points['total_score'] = total_points['total_points_home'] + total_points['total_points_away']

fig = px.bar(total_points.sort_values(by='total_score', ascending= False).head(20),
             x='country',
             y=['total_points_home', 'total_points_away','total_score'],
             labels={'value': 'Points', 'variable': 'Category'},
             title='Total Points, Home Team Points, and Away Team Points by Country',
             )

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Longest win streaks and lose streaks

In [16]:
max_streaks = {}

for team in pd.concat([results_['home_team'], results_['away_team']]).unique():
    team_matches = results_[(results_['home_team'] == team) | (results_['away_team'] == team)]
    
    current_streak = 0
    current_streak_type = None
    max_streak = 0
    max_streak_type = None
    
    for i, row in team_matches.iterrows():
        if row['winner'] == team:
            if current_streak_type == 'win':
                current_streak += 1
            else:
                current_streak = 1
                current_streak_type = 'win'
        elif row['winner'] == 'tie':
            current_streak = 0
            current_streak_type = None
        else:
            if current_streak_type == 'lose':
                current_streak += 1
            else:
                current_streak = 1
                current_streak_type = 'lose'
        
        if current_streak > max_streak:
            max_streak = current_streak
            max_streak_type = current_streak_type
    
    max_streaks[team] = (max_streak, max_streak_type)

win_streaks_df = pd.DataFrame([(team, streak, streak_type) for team, (streak, streak_type) in max_streaks.items() if streak_type == 'win'], columns=['team', 'streak', 'type'])
streaks_df = win_streaks_df.sort_values('streak', ascending=False).head(10)

fig = px.bar(streaks_df, 
             x='team', 
             y='streak', 
             title='Top 10 Teams with Longest Win Streaks', 
             labels={'team':'Team', 'streak':'Win Streak'}, 
             color='team',
             text = 'streak')
fig.update_layout(showlegend=False)

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [17]:
lose_streaks_df = pd.DataFrame([(team, streak, streak_type) for team, (streak, streak_type) in max_streaks.items() if streak_type == 'lose'], columns=['team', 'streak', 'type'])
streaks_df = lose_streaks_df.sort_values('streak', ascending=False).head(10)

fig = px.bar(streaks_df, 
             x='team', 
             y='streak', 
             title='Top 10 Teams with Longest Lose Streaks', 
             labels={'team':'Team', 'streak':'Lose Streak'}, 
             color='team',
             text = 'streak')
fig.update_layout(showlegend = False)
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



GOALS SCORED and CONCEDED

In [18]:
home_team_stats = results_.groupby('home_team').agg({'home_score': 'sum', 'away_score': 'sum'}).reset_index()
home_team_stats.columns = ['team', 'goals_scored', 'goals_conceded']

away_team_stats = results_.groupby('away_team').agg({'away_score': 'sum', 'home_score': 'sum'}).reset_index()
away_team_stats.columns = ['team', 'goals_scored', 'goals_conceded']

team_stats = home_team_stats.merge(away_team_stats, on='team', how='outer', suffixes=('_home', '_away'))

# Fill missing values with 0 (teams that have only played as home or away)
team_stats = team_stats.fillna(0)

team_stats['total_goals_scored'] = team_stats['goals_scored_home'] + team_stats['goals_scored_away']
team_stats['total_goals_conceded'] = team_stats['goals_conceded_home'] + team_stats['goals_conceded_away']
teams_goals_scored = team_stats.sort_values(by = "total_goals_scored" , ascending = False).head(10)

In [19]:
fig = px.bar(teams_goals_scored,
            x= 'team',
            y= 'total_goals_scored',
            color = 'team',
            text = 'total_goals_scored',
            height = 400)
fig.update_layout(title = 'Top 10 teams by goals scored',
                  showlegend = False)
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [20]:
teams_goals_conceded = team_stats.sort_values(by = 'total_goals_conceded', ascending = False).head(10)

fig = px.bar(teams_goals_conceded,
            x= 'team',
            y= 'total_goals_conceded',
            color = 'team',
            text = 'total_goals_conceded',
            height = 400)
fig.update_layout(title = 'Top 10 teams by goals conceded',
                  showlegend = False)
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



MAJOR TOURNAMENTS

In [21]:
major_tournaments = results[
   (results.tournament == 'FIFA World Cup') |
   (results.tournament == 'UEFA Nations League')|
   (results.tournament == 'Copa América')|
   (results.tournament == 'UEFA Euro')|
   (results.tournament == 'African Cup of Nations')|
   (results.tournament == 'Confederations Cup')|
   (results.tournament == "King's Cup")|
   (results.tournament == 'AFC Asian Cup')
  ]

major_tournaments.sample(3)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,Year,Month,day,week_day,winner,gap
15557,1988-12-08,Iran,United Arab Emirates,1,0,AFC Asian Cup,Doha,Qatar,True,1988,12,8,Thu,Iran,1
771,1923-12-02,Uruguay,Argentina,2,0,Copa América,Montevideo,Uruguay,False,1923,12,2,Sun,Uruguay,2
2106,1938-06-05,Italy,Norway,2,1,FIFA World Cup,Marseille,France,True,1938,6,5,Sun,Italy,1


Win ratio in major tournaments for teams with more than 100 games

In [22]:
team_stats = pd.DataFrame()

team_stats['Total games'] = pd.concat([major_tournaments['home_team'], major_tournaments['away_team']]).value_counts()
team_stats['Wins'] = major_tournaments['winner'][major_tournaments['winner'] != 'tie'].value_counts()
team_stats['Tie'] =  major_tournaments['winner'][major_tournaments['winner'] == 'tie'].value_counts()
team_stats['Win ratio'] = team_stats['Wins'] / team_stats['Total games']

team_stats = team_stats[team_stats['Total games'] > 100].reset_index().sort_values('Win ratio', ascending = False)

fig = px.bar(team_stats[:10],
             x= 'index',
             y= 'Win ratio',
             title= 'Win ratio in major tournaments',
             text_auto= True)

fig.show()
print('The fiercest rivalry: Brazil vs Argentina stand on top')
print('In this list, there are 4 European and 3 African teams')

The fiercest rivalry: Brazil vs Argentina stand on top
In this list, there are 4 European and 3 African teams


In [23]:
major_tournaments.loc[:, 'team_pair']= major_tournaments['home_team'] + ' vs ' + major_tournaments['away_team']

team_pair_counts = major_tournaments.groupby('tournament')['team_pair'].value_counts().reset_index(name='occurrences')
most_occurred_team_pairs = team_pair_counts.groupby('tournament').apply(lambda x: x.loc[x['occurrences'].idxmax()]).reset_index(drop=True).sort_values(by='occurrences', ascending = False)

fig = px.bar(most_occurred_team_pairs, 
             x='occurrences', 
             y='tournament', 
             color='team_pair',
             title='Most Occurred Team Pairs by Tournament',
             labels={'occurrences': 'Occurrences', 'pair': 'Team Pair'},
             text = 'team_pair',
             orientation = 'h'
             )

fig.update_layout(showlegend = False)
fig.show()

/tmp/ipykernel_18/3510857707.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_18/3510857707.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Clean sheet

In [24]:
import numpy as np
clean_sheets = (major_tournaments.assign(home_clean_sheet = major_tournaments['away_score'] == 0,
                                    away_clean_sheet = major_tournaments['home_score'] == 0)
                .melt(id_vars=['home_team', 'away_team'], 
                      value_vars=['home_clean_sheet', 'away_clean_sheet'])
                .rename(columns={'variable': 'location', 'value': 'clean_sheet'})
                .assign(team = lambda df: np.where(df['location'] == 'home_clean_sheet', df['home_team'], df['away_team']))
                .groupby('team')['clean_sheet'].sum()
                .reset_index()
                .rename(columns={'clean_sheet': 'total_clean_sheets'})
                .sort_values(by='total_clean_sheets', ascending=False))

fig = px.bar(clean_sheets.head(10),
             x= 'total_clean_sheets',
             y='team',
             orientation='h',
             text = 'total_clean_sheets')

fig.update_layout(title = 'Top 10 teams with most clean sheet matches')
fig.show()

## Goalscorers dataframe

In [25]:
scorers.sample(3)

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
6358,1973-10-20,Italy,Switzerland,Italy,Gianni Rivera,39.0,False,True
33859,2015-06-16,North Korea,Uzbekistan,North Korea,Ro Hak-su,34.0,False,False
35494,2016-10-06,Saudi Arabia,Australia,Saudi Arabia,Nasser Al-Shamrani,79.0,False,False


In [26]:
scorers.minute.isna().sum()

260

In [27]:
scorers_df = scorers[scorers['own_goal'] == False]

best_scorers = scorers_df.groupby('scorer').size().reset_index(name = 'total goals')
most_penalty = scorers_df.groupby('scorer')['penalty'].sum().reset_index(name = 'total penalties')

combined_data = pd.merge(best_scorers, most_penalty, on='scorer', how='outer').fillna(0).sort_values('total goals', ascending = False)[:10]

fig = go.Figure()
fig.add_trace(go.Bar(x=combined_data['scorer'], y=combined_data['total goals'], name='Total Goals'))
fig.add_trace(go.Bar(x=combined_data['scorer'], y=combined_data['total penalties'], name='Total Penalties'))

fig.update_layout(title = 'TOP 10 GOAL SCORERS')
fig.show()

Goals scored per minute

In [28]:
goals_per_minute = scorers_df['minute'].value_counts().reset_index(name = 'goals').sort_values(by = 'minute', ascending = True)

# Combine goals after the 90th minute into the number of goals in the 90th minute
goals_per_minute['goals'] = goals_per_minute.apply(lambda row: row['goals'] + goals_per_minute[goals_per_minute['minute'] > 90]['goals'].sum() if row['minute'] == 90 else row['goals'], axis=1)
goals_per_minute = goals_per_minute[goals_per_minute['minute'] <= 90]
goals_per_minute

,minute,goals
89,1.0,192.0
86,2.0,328.0
87,3.0,307.0
85,4.0,333.0
79,5.0,375.0
...,...,...
22,86.0,501.0
12,87.0,516.0
3,88.0,587.0
2,89.0,664.0


In [29]:
fig = px.bar(goals_per_minute,
              x= 'minute',
              y = 'goals',
              title = 'Goals scored per Minute',
              text = 'goals')

fig.show()

Goals scored per 15 minute

In [30]:
scorers_df['minute_group'] = ((scorers_df['minute'] // 15) + 1) * 15

goals_per_15_minutes = scorers_df.groupby(['team', 'minute_group'])['scorer'].count().reset_index()
goals_pivot = goals_per_15_minutes.pivot(index='team', columns='minute_group', values='scorer').fillna(0).astype(int).reset_index()

selected_teams = ["Argentina", "Brazil", "Germany", "France", "Italy", "Spain", "Netherlands", "Portugal", "England", "Belgium",
                  "Thailand", "Vietnam"]
                
heat_map = goals_pivot[goals_pivot['team'].isin(selected_teams)]

#Heat map
fig = px.imshow(heat_map.iloc[:, 2:], 
                x=heat_map.columns[2:], 
                y=heat_map['team'], 
                labels=dict(y='Team', x='15-Minute Intervals', color='Goals Scored'),
                title='Goals Scored per 15 Minutes by Team',
                color_continuous_scale='YlOrRd',
                text_auto =True
                )

fig.show()

/tmp/ipykernel_18/4039479508.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Players who scored the most in the 1st half and 2nd half

In [31]:
first_half = scorers_df[scorers.minute <= 45]
second_half = scorers_df[(scorers.minute >45) & (scorers.minute <=90)]

top_scorer_1st_half = first_half.groupby('scorer').size().reset_index(name = 'goals').nlargest(columns = 'goals',n=3)
top_scorer_2nd_half = second_half.groupby('scorer').size().reset_index(name = 'goals').nlargest(columns = 'goals', n=3)

combined = pd.concat([top_scorer_1st_half.assign(Half = '1st half'), 
                      top_scorer_2nd_half.assign(Half = '2nd half')
                      ])

fig = px.bar(combined, x='Half', y='goals',
             color = 'goals',
             title='Top Scorers in the 1st and 2nd Halves', 
             labels={'goals': 'Number of Goals'}, 
             category_orders={"Half": ["1st Half", "2nd Half"]},
             text = 'scorer')

fig.update_layout(barmode = 'group', showlegend = False)
fig.show()

/tmp/ipykernel_18/2670137952.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_18/2670137952.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Players who have most hattricks

In [32]:
scorers_df['match_id'] = scorers_df['date'].astype(str) + scorers_df['home_team'] + scorers_df['away_team']

player_goals = scorers_df.groupby(['scorer', 'match_id']).size()
hattricks  = player_goals[player_goals >=3]

player_hattricks = hattricks.groupby('scorer').size().reset_index(name = 'number of hattricks').sort_values(by='number of hattricks', ascending=False)


fig = px.bar(player_hattricks.head(10), x='scorer', y='number of hattricks',
             text_auto= True,
             title ='Top 10 players with most hattricks')
fig.show()
print('This data is missing many goals Messi scored for Argentina!')

/tmp/ipykernel_18/4011709737.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



This data is missing many goals Messi scored for Argentina!


Favorite opponents

In [33]:
scorers_df['opponent'] = scorers_df.apply(lambda row: row['away_team'] if row['team'] == row['home_team'] else row['home_team'], axis = 1)

fav_opp = scorers_df.groupby(['scorer', 'opponent']).size().reset_index(name='goals')
fav_opp = fav_opp.loc[fav_opp.groupby('scorer')['goals'].idxmax()]

players = ['Lionel Messi', 'Romelu Lukaku', 'Cristiano Ronaldo', 'Robert Lewandowski', 'Neymar', 'Harry Kane', 'Diego Maradona', 'Kylian Mbappé',
           'Olivier Giroud', 'Thomas Müller', 'Pelé', 'Son Heung-min', 'Lê Công Vinh','Luis Suárez',
           'Edin Džeko', 'Miroslav Klose']
top_players_stats = fav_opp[fav_opp['scorer'].isin(players)]
top_players_stats['label'] = top_players_stats['opponent'] + ': ' + top_players_stats['goals'].astype(str) + ' goals'

fig = px.bar(top_players_stats, x='goals',
             y= 'scorer',
             color = 'goals',
             title = 'Goals scored against favorite opponents by famous goalscorers',
             text = 'label',
             orientation = 'h')
            
fig.show()


/tmp/ipykernel_18/253869649.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_18/253869649.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Own goals

In [34]:
own_goals_df = scorers[scorers.own_goal == True]

In [35]:
own_goals_df.groupby('scorer').size().nlargest(n=10)

scorer
Cristian Brolli           3
Gustavo Gómez             3
José del Solar            3
Ján Ďurica                3
Ragnar Klavan             3
Raio Piiroja              3
Rashad Sadygov            3
Roberto Ayala             3
Walid Abbas               3
Alessandro Della Valle    2
dtype: int64

In [36]:
own_goals_df.groupby('team').size().nlargest(n=10)

team
England    23
Spain      23
Germany    21
France     19
Russia     19
Brazil     18
Italy      18
Belgium    17
Mexico     16
Hungary    14
dtype: int64

In [37]:
own_goals_per_minute= own_goals_df['minute'].value_counts().reset_index(name = 'own goals').sort_values(by = 'minute', ascending = True)

fig = px.bar(own_goals_per_minute,
            x='minute',
             y = 'own goals',
             title = 'Owned goals per Minute')

fig.show()

## Shootout dataframe

In [38]:
shootouts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           562 non-null    object
 1   home_team      562 non-null    object
 2   away_team      562 non-null    object
 3   winner         562 non-null    object
 4   first_shooter  86 non-null     object
dtypes: object(5)
memory usage: 22.1+ KB


In [39]:
shootouts.first_shooter.isna().sum()

476

Best teams and worst teams in Penalty shootouts

In [40]:
most_penalty_wins = shootouts.groupby('winner').size().nlargest(n=10).reset_index(name = 'wins')

fig = px.bar(most_penalty_wins,
             x= 'winner',
             y = 'wins',
             title = 'Most penalty shootout wins',
             text = 'wins')

fig.show()

In [41]:
shootouts['loser'] = shootouts.apply(lambda row: row['home_team'] if row['winner'] == row['away_team'] else row['away_team'], axis=1)

most_penalty_loss = shootouts.groupby('loser').size().nlargest(n=10).reset_index(name = 'loss')

fig = px.bar(most_penalty_loss,
             x= 'loser',
             y = 'loss',
             title = 'Most penalty shootout loss',
             text = 'loss'
             )

fig.update_layout(showlegend = False)
fig.show()

Teams confronting each other most

In [42]:
shootouts['pair'] = shootouts['home_team'] + " vs " + shootouts['away_team']
most_occurred_pairs = shootouts.groupby('pair').size().nlargest(n=10).reset_index(name = 'matches')
most_occurred_pairs

,pair,matches
0,Kenya vs Uganda,4
1,Brazil vs Paraguay,3
2,Japan vs South Korea,3
3,Mali vs Guinea,3
4,United States vs Panama,3
5,Uruguay vs Brazil,3
6,Zambia vs Namibia,3
7,Argentina vs Brazil,2
8,Argentina vs Colombia,2
9,Botswana vs South Africa,2
